# Google Earth Engine Raster Download

This script can be used to download data from GEE.

**Original code:** [Neeraj Baruah](https://www.vivideconomics.com/neeraj-baruah/) & [Konstantinos Pegios](https://github.com/kopegios) <br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [1]:
#Import modules and libraries
#Information on installation are available here https://developers.google.com/earth-engine/python_install_manual
#for Windows 10 a simmple < pip install earthengine-api > shall do the work

import ee
from ee import batch
import time

### Define the boundaries and the target coordinate systems

In [2]:
## Use of bboxfinder - http://bboxfinder.com/#6.293459,0.856934,12.425848,3.911133 in WGS84

bounds = [0.856934,6.293459,3.911133,12.425848]
output_resolution = 1000     # in m
output_crs = 'EPSG:32631'   # suggested projection systems for Benin

#### Example of extracting single images

In [3]:
ee.Initialize()

#Function to extract GEE Images
def gee_extract_Image(image, band, desc, output_scale, bounds, crs): 
    
    aoi = ee.Geometry.Rectangle(bounds)
    aoi = aoi['coordinates'][0]
    print("Bounds created. Reading in collection ...")

    dataset = ee.Image(image).select(band)
    
    print("Image extracted...")
    out = batch.Export.image.toDrive(image=dataset, description=desc, 
                                     scale=output_scale, crs=crs, region =aoi)
    process = batch.Task.start(out)
    print("Initiating download to Google Drive")


## Define layers to be extracted and downloaded to your Google Drive

images = [                                                                                      \
          ['USGS/GFSAD1000_V0', 'landcover', 'GFSAD-landcover', output_resolution, bounds, output_crs], \
          ['CGIAR/SRTM90_V4', 'elevation', 'SRTM-elevation', output_resolution, bounds, output_crs],   \
         ]

start = time.time()

for i in images:
    print("Processing {0} ..." .format(i[1]))
    gee_extract_Image(i[0], i[1], i[2], i[3], i[4], i[5])

end = time.time()
print("Processed in {0}s" .format(round(end - start,2)))



Processing landcover ...
Bounds created. Reading in collection ...
Image extracted...
Initiating download to Google Drive
Processing elevation ...
Bounds created. Reading in collection ...
Image extracted...
Initiating download to Google Drive
Processed in 1.19s


#### Example of extracting images under time series

In [4]:
ee.Initialize()

#Function to extract GEE image collections
def gee_extract_ImageCollection(collection, time1, time2, band, desc, output_scale, bounds, crs):     
    
    aoi = ee.Geometry.Rectangle(bounds)
    print("Bounds created. Reading in collection ...")
    dataset = ee.ImageCollection(collection)
    dataset_time = dataset.filterDate(time1, time2)
    bands = dataset_time.select(band)  ## another function for multiple bands - this is for single band only
    composite = bands.median().clip(aoi);   ### Can use different reducer or simpleComposite algorithm
    print("Image composite created ...")
    #print(composite.getInfo())
    
    out = batch.Export.image.toDrive(image=composite, description=desc,scale=output_scale, crs=crs)
    process = batch.Task.start(out)
    print("Initiating download to Google Drive...")

'''
##############################
##############################
#######                #######
#######    VARIABLES   #######
#######                #######
##############################
############################## 
'''

'''Input parameters as a list:
       @dataset-collection
       @beginning of compoiste time range
       @end of composite time range
       @band
       @description
       @output spatial resolution
       @area of interest bounds'''
      
imageCollections = [                                                                                      \
                    ['MODIS/006/MOD13Q1', '2018-01-01', '2018-12-31', 'NDVI', 'modis-ndvi', output_resolution, bounds, output_crs], \
                    ['MODIS/006/MOD13Q1', '2018-01-01', '2018-12-31', 'EVI', 'modis-evi', output_resolution, bounds, output_crs], \
                    ['MODIS/006/MOD09GA', '2018-01-01', '2018-12-31', 'sur_refl_b02', 'MODIS-Band2', output_resolution, bounds, output_crs], \
         ]

start = time.time()
for i in imageCollections:
    print("Processing {0} ..." .format(i[4]))
    gee_extract_ImageCollection(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7])

end = time.time()
print("Processed in {0}s" .format(round(end - start,2)))


Processing modis-ndvi ...
Bounds created. Reading in collection ...
Image composite created ...
Initiating download to Google Drive...
Processing modis-evi ...
Bounds created. Reading in collection ...
Image composite created ...
Initiating download to Google Drive...
Processing MODIS-Band2 ...
Bounds created. Reading in collection ...
Image composite created ...
Initiating download to Google Drive...
Processed in 1.7s
